## CustomTrainerVietClassifier
- model.fit()에서 벗어나기 PJT

In [1]:
import os
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# training set과 test set의 모든 이미지 파일에 대해서,
# jpg image header가 포함되지 않은 (jpg의 파일 구조에 어긋나는) 파일들을 삭제해줍니다.

data_path = '/aiffel/aiffel/model-fit/data/30vnfoods/'
train_path = data_path + 'Train/'
test_path = data_path + 'Test/'

for path in [train_path, test_path]:
    classes = os.listdir(path)

    for food in classes:
        food_path = os.path.join(path, food)
        images = os.listdir(food_path)
        
        for image in images:
            with open(os.path.join(food_path, image), 'rb') as f:
                bytes = f.read()
            if bytes[:3] != b'\xff\xd8\xff':
                print(os.path.join(food_path, image))
                os.remove(os.path.join(food_path, image))

In [3]:
classes = os.listdir(train_path)
train_length = 0

for food in classes:
    food_path = os.path.join(train_path, food)
    images = os.listdir(food_path)
    
    train_length += len(images)

print('training data의 개수: '+str(train_length))

training data의 개수: 9775


In [4]:
# 문제1: dataloader 구현하기

def process_path(file_path, class_names, img_shape=(224, 224)):
    '''
    file_path로 부터 class label을 만들고, 이미지를 읽는 함수
    이미지 크기를 (224, 224)로 맞춰주세요.
    '''
    label = tf.strings.split(file_path, os.path.sep)
    label = label[-2] == class_names
    label = tf.cast(label, tf.float32)

    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, img_shape) 

#     [keypoint,] = tf.py_function(process_keypoint, [file_path], [tf.float32])
    
    return img, label

def prepare_for_training(ds, batch_size=32, cache=True, shuffle_buffer_size=1000):
    '''
    TensorFlow Data API를 이용해 data batch를 만드는 함수
    '''
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat(2)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

def load_data(data_path, batch_size=32):
    '''
    데이터를 만들기 위해 필요한 함수들을 호출하고 데이터를 리턴해주는 함수
    TensorFlow Dataset 객체를 생성하고 process_path 함수로 이미지와 라벨을 묶은 다음,
    prepare_for_training 함수로 batch가 적용된 Dataset 객체를 만들어주세요.
    '''
    class_names = [cls for cls in os.listdir(data_path) if cls != '.DS_Store']
    data_path = pathlib.Path(data_path)

    list_ds = tf.data.Dataset.list_files(str(data_path/'*/*'))
    labeled_ds = list_ds.map(lambda x: process_path(x, class_names, img_shape=(224, 224)))
    ds = prepare_for_training(labeled_ds, batch_size=batch_size)

    return ds

In [5]:
# 문제2: 모델 구현하기

from tensorflow.keras.applications import EfficientNetB0

class Model(tf.keras.Model):
    '''
    EfficientNetB0을 백본으로 사용하는 모델을 구성합니다.
    Classification 문제로 접근할 것이기 때문에 맨 마지막 Dense 레이어에 
    우리가 원하는 클래스 개수만큼을 지정해주어야 합니다.
    '''
    def __init__(self, num_classes=5, freeze=False):
        super(Model, self).__init__()
        self.base_model = EfficientNetB0(include_top=False, weights='imagenet')
        if freeze:
            self.base_model.trainable = False
        self.top = tf.keras.Sequential([tf.keras.layers.GlobalAveragePooling2D(name="avg_pool"),
                                       tf.keras.layers.BatchNormalization(),
                                       tf.keras.layers.Dropout(0.5, name="top_dropout")])
        self.classifier = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")
    def call(self, inputs, training=True):
        x = self.base_model(inputs)
        x = self.top(x)
        x = self.classifier(x)
        return x
    
if __name__ == '__main__':
    model = Model(num_classes=5, freeze=True)
    model.build(input_shape=(None, 224, 224, 3))
    print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
sequential (Sequential)      (None, 1280)              5120      
_________________________________________________________________
pred (Dense)                 multiple                  6405      
Total params: 4,061,096
Trainable params: 8,965
Non-trainable params: 4,052,131
_________________________________________________________________
None


- EfficientNetB0 backbone과 Dense 레이어를 결합하여 모델 구현

In [6]:
# 문제3: custom trainer 구현하기

class Trainer:
    def __init__(self, model, epochs, batch, loss_fn, optimizer):
        self.model = model
        self.epochs = epochs
        self.batch = batch
        self.loss_fn = loss_fn
        self.optimizer = optimizer
    def train(self, train_dataset, train_metric):
        for epoch in range(self.epochs):
            print("\nStart of epoch %d" % (epoch+1,))
            # 매 batch 마다 반복적으로 학습
            for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
                with tf.GradientTape() as tape:
                    logits = model(x_batch_train, training=True)
                    loss_value = self.loss_fn(y_batch_train, logits)
                grads = tape.gradient(loss_value, model.trainable_weights)
                self.optimizer.apply_gradients(zip(grads, model.trainable_weights))
                # train metric 업데이트
                train_metric.update_state(y_batch_train, logits)
                # 5 배치마다 로깅
                if step % 16 == 0:
                    print(
                        "Training loss (for one batch) at step %d: %.4f"
                        % (step, float(loss_value))
                    )
                    print("Seen so far: %d samples" % ((step + 1) * self.batch))
                    print(train_metric.result().numpy())

            # 마지막 epoch 학습이 끝나면 train 결과를 보여줌
            train_acc = train_acc_metric.result()
            print("Training acc over epoch: %.4f" % (float(train_acc),))

In [7]:
# 모델 학습 코드

train_path = "/aiffel/aiffel/model-fit/data/30vnfoods/Train"

epoch = 5
batch = 32

model = Model(num_classes=10)
dataset = load_data(data_path=train_path, batch_size=batch)
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
trainer = Trainer(model=model,
                epochs=epoch,
                batch=batch,
#                 ds_length=train_length,
                loss_fn=loss_function,
                optimizer=optimizer)

trainer.train(train_dataset=dataset,
            train_metric=train_acc_metric)


Start of epoch 1
Training loss (for one batch) at step 0: 3.8146
Seen so far: 32 samples
0.125
Training loss (for one batch) at step 16: 1.6550
Seen so far: 544 samples
0.3602941
Training loss (for one batch) at step 32: 1.4073
Seen so far: 1056 samples
0.47916666
Training loss (for one batch) at step 48: 0.6260
Seen so far: 1568 samples
0.53635204
Training loss (for one batch) at step 64: 1.0460
Seen so far: 2080 samples
0.5692308
Training loss (for one batch) at step 80: 1.1010
Seen so far: 2592 samples
0.5952932
Training loss (for one batch) at step 96: 1.3434
Seen so far: 3104 samples
0.61533505
Training loss (for one batch) at step 112: 0.4797
Seen so far: 3616 samples
0.63274336
Training loss (for one batch) at step 128: 1.1150
Seen so far: 4128 samples
0.6455911
Training loss (for one batch) at step 144: 0.4183
Seen so far: 4640 samples
0.6599138
Training loss (for one batch) at step 160: 0.7884
Seen so far: 5152 samples
0.67177796
Training loss (for one batch) at step 176: 0.9

- 학습이 진행되면서 training accuracy가 점차 증가

In [8]:
# 모델 테스트 코드

test_ds = load_data(data_path=test_path)

for step_train, (x_batch_train, y_batch_train) in enumerate(test_ds.take(10)):
    prediction = model(x_batch_train)
    print("{}/{}".format(np.array(tf.equal(tf.argmax(y_batch_train, axis=1), tf.argmax(prediction, axis=1))).sum(), tf.argmax(y_batch_train, axis=1).shape[0]))

27/32
27/32
26/32
31/32
29/32
29/32
30/32
27/32
26/32
29/32


- 커스텀 데이터를 커스텀 트레이너로 학습

### 회고

#### 문제 해결
- JPEG 파일의 유효성 검증을 통해 데이터 품질 보장하여 훈련 과정 중에 발생할 수 있는 예기치 않은 오류 미연 방지
- 데이터 로더 구현에서 TensorFlow의 Data API를 활용하여 효율적으로 대용량 데이터를 처리할 수 있는 파이프라인을 구축. AUTOTUNE을 활용한 프리패치 기능으로 학습 속도 향상
- 모델 구성에서 EfficientNetB0 사용하여 특성 추출 기능 활용하고, 최종 출력 레이어를 커스터마이징하여 문제에 적합하게 조정

#### 배운 점:
- TensorFlow Data API를 활용한 데이터 전처리 및 모델 공급 방법 학습
- EfficientNetB0을 백본으로 사용한 transfer learning 수행 방법 습득
- Custom trainer 구현을 통한 학습 loop 내부 동작 이해

#### 발전시킬 점: 
- 큰 데이터셋 처리를 위해 tf.data.Dataset의 from_generator()나 from_tensor_slices()를 활용한 lazy loading 적용
- Adam이나 RMSProp 등 발전된 optimizer 적용 고려
- 정확도 개선을 위한 데이터 확장(augmentation) 기법 적용
- 보다 깊은 backbone 네트워크 사용 검토
- Learning rate scheduler 도입을 통한 학습률 최적화 시도

#### 결과 해석:
- 테스트 결과 준수한 성능이 나왔으나, 실제 서비스 활용을 위해서는 정확도 개선을 더 진행할 수 있을 것으로 보임
- 10개의 테스트 배치에서 평균적으로 배치당 정답 개수가 전체 개수 평균 정확도: 87.81%